In [ ]:
import sys
# from product import *
from data_util.product import *

import pandas as pd
import random
import time
import re

import os
import hashlib
import struct
import subprocess
import collections
import tensorflow as tf
from tensorflow.core.example import example_pb2
import nltk

import sys
import shutil
import tqdm
import random

from copy import deepcopy
# from product import *

VOCAB_SIZE = 50000
CHUNK_SIZE = 1000  # num examples per chunk, for the chunked data

# Key word Attention DataSet 讀取

In [ ]:
# from data_util.config import *
# _, category1, category2, _ = DVD_Player().getAttr()
_,category1,category2,_ = Cameras().getAttr()
# _,category1,category2,_ = Cell_Phones().getAttr()
# _,category1,category2,_ = GPS().getAttr()
# _,category1,category2,_ = Keyboards().getAttr()
# category1,category2 = config.category1 , config.category2

xlsx_path = "XLSX/category/%s_%s_key.xlsx"%(category1,category2)
# df.to_csv(csv_path) #默认dt是DataFrame的一个实例，参数解释如下
# key_train_df.to_excel(csv_path, encoding='utf8')
orign_key_df = pd.read_excel(xlsx_path)
print(xlsx_path + " Read finished")
len(orign_key_df)

orign_key_df['bert_review'] = '' 
orign_key_df['bert_summary'] = ''
orign_key_df['bert_review_len'] = 0
orign_key_df['bert_summary_len'] = 0

orign_key_df.head()


# Key word load

In [ ]:
fn = 'FOP-View/%s_%s_keywords2.txt' % (category1, category2)
print('load %s keywords...' % (fn))
total_keywords = set()
with open(fn, 'r', encoding='utf-8') as f:
    lines = f.readlines()
    for line in lines:
        k, v = line.split(":")
        total_keywords.add(k)

# Total Opinion

In [ ]:
opinion_lexicon = {}
for filename in os.listdir('opinion-lexicon-English/'):      
    if "txt" not in filename: continue
    print(filename)
    with open('opinion-lexicon-English/'+filename,'r') as f_input:
        lexion = []
        for line in f_input:
            if line.startswith(";"):
                continue
            word = line.replace("\n","")
            if word != "" : lexion.append(word)
        pos = filename.replace(".txt","")
        opinion_lexicon[pos] = lexion

opinion_lexicon["total-words"] = opinion_lexicon["negative-words"] + opinion_lexicon["positive-words"]
print("total-words 已取得")

In [ ]:
# tokenizer 裡頭的字典資訊

In [ ]:
from pytorch_pretrained_bert import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True, do_basic_tokenize=True)
vocab = tokenizer.vocab # word_to_id
print("字典大小：", len(vocab))

# Bert-Summary 資料清理

In [ ]:
import spacy
from collections import Counter
import en_core_web_sm
from spacy.tokenizer import Tokenizer
from spacy.lang.en import English

nlp = en_core_web_sm.load()

def isnumber(aString):
    try:
        float(aString)
        return True
    except:
        return False
    
def create_custom_tokenizer(nlp):
    prefix_re = re.compile(r'[0-9]\.')
    return Tokenizer(nlp.vocab, prefix_search=prefix_re.search)

nlp.tokenizer = create_custom_tokenizer(nlp)

alphbet_stopword = ['b','c','d','e','f','g','h','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']

In [ ]:
def compose_summary(x):
    #x = x.replace("\n"," ")
    x = x.replace("\n","").replace("</s>","").replace("<s>","")
    x = "<s>" + x + "</s>"  
    tokens = [str(token) for token in x.split(" ") if (" " not in str(token)) and \
                   #(str(token).isalpha()) and \
                  (len(str(token)) > 1)   ]
    # print(tokens)
    #return " ".join(tokens),tokens
    
    newtokens = []
    for token in tokens:
        if (isnumber(token) or len(token) == 1 or token == ".") and (token not in alphbet_stopword):
            newtokens.append(token)
        else:
            token = token.replace("."," . ")
            sub_tokens = token.split(" ")
            sub_tokens = [t for t in sub_tokens if t != "" and t not in alphbet_stopword]
            newtokens.extend(sub_tokens)
    
#     dot_tokens = [token for token in newtokens if (token[0] == "." or token[-1] == ".") and (len(token)>1)]
#     if len(dot_tokens) > 0 :print(dot_tokens)
        
    return " ".join(newtokens).replace(' . . ',' . '),newtokens
    

def bert_compose_summary(newtokens):
#     x = x.replace("\n","").replace("</s>","").replace("<s>","")
#     x = "<s>" + x + "</s>"  
#     tokens = [str(token) for token in nlp(x) if (" " not in str(token)) and \
#                   (str(token).isalpha()) and \
#                   (len(str(token)) > 1)   ]
    
#     newtokens = []
#     for token in tokens:
#         if (isnumber(token) or len(token) == 1 or token == ".") and (token not in alphbet_stopword):
#             newtokens.append(token)
#         else:
#             token = token.replace("."," . ")
#             sub_tokens = token.split(" ")
#             sub_tokens = [t for t in sub_tokens if t != "" and t not in alphbet_stopword]
#             newtokens.extend(sub_tokens)

#     dot_tokens = [t for t in newtokens if ("." in t) and (len(t) > 1) ]
#     if len(dot_tokens) > 0 :print(dot_tokens)
    newtokens = [t for t in newtokens if t not in ["<s>","</s>"]]
    newtokens = ['[CLS]'] + tokenizer.tokenize(" ".join(newtokens)) + ['[SEP]']
    return " ".join(newtokens)

'''
def calc_summary_len(x):
#     tokens = [token for token in nlp(x)]
#     print(tokens)
#     print([len(t) for t in tokens])
#     return len(tokens)
    return len(x.split(" "))

nlp.tokenizer = create_custom_tokenizer(nlp)

orign_key_df['lemm_summary'] = orign_key_df['lemm_summary'].apply(compose_summary)
orign_key_df['lemm_summary_len'] = orign_key_df['lemm_summary'].apply(calc_summary_len)


amount = len(orign_key_df)
print('Total data : %s'%(amount))

orign_key_df.head()
'''
orign_key_df.head()

In [9]:
from tqdm import tqdm
# 非符號alpha word重疊數
with tqdm(total=len(orign_key_df)) as pbar:
    for i ,row in orign_key_df.iterrows():       
        lemm_summary,newtokens = compose_summary(row['lemm_summary'])
        bert_summary = bert_compose_summary(newtokens)
        
        orign_key_df.loc[i,'lemm_summary'] = lemm_summary
        orign_key_df.loc[i,'bert_summary'] = bert_summary
        
        orign_key_df.loc[i,'lemm_summary_len'] = len(lemm_summary.split(" "))       
        orign_key_df.loc[i,'bert_summary_len'] = len(bert_summary.split(" "))

        pbar.update(1)
        
amount = len(orign_key_df)
print('Total data : %s'%(amount))
orign_key_df.head()

100%|██████████| 154488/154488 [1:45:58<00:00, 24.30it/s]Total data : 154488



,review_ID,review,summary,big_categories,small_categories,lemm_review,lemm_summary,lemm_review_len,lemm_summary_len,overall,vote,total_keyword,FOP_sents,total_mention_features,bert_review,bert_summary,bert_review_len,bert_summary_len
0,1356566400,this book was almost as small a pamphlet for 2...,too small,Electronics,Camera Cases,['this book was almost as small a pamphlet for...,<s> too small </s>,44,4,1,2,NaN,NaN,intend small book pamphlet gift expect emphasi...,,[CLS] too small [SEP],0,4
1,1325721600,this book is a collectors item and i love they...,wonderful book a keepsake,Electronics,Camera Cases,['this book is a collector item and i love the...,<s> wonderful book keepsake </s>,53,5,5,9,"item wonderful,quality item,nostalgia wonderfu...",great quality though and a wonderful nostalgia...,item collector book price copy high buy price ...,,[CLS] wonderful book keeps ##ake [SEP],0,6
2,951782400,i would like to say the the cannon elph37oz is...,elph 37oz is the best,Electronics,APS Cameras,['i would like to say the the cannon elph37 oz...,<s> elph 37 oz is the best </s>,58,8,5,51,"camera inferior,money extra,camera cheap",pay the extra money and get this one rather th...,cannon money camera buy decide camera picture ...,,[CLS] el ##ph 37 oz is the best [SEP],0,9
3,985910400,i have had and used q 370z for over a year and...,a good all around aps camera,Electronics,APS Cameras,['i have have and use q 370z for over a year a...,<s> good all around aps camera </s>,137,7,5,13,"eye red,light low room increase lense mm,slrs ...",yes i sometimes get red eye in low light but a...,snapshot photographer year low lens red avoid ...,,[CLS] good all around ap ##s camera [SEP],0,8
4,967075200,i used this camera once and it took relatively...,terrible camera worse customer service,Electronics,APS Cameras,['i use this camera once and it take relativel...,<s> terrible camera worse customer service </s>,96,7,1,48,memory internal camera service camera send sty...,however on my next trip the camera s internal ...,camera picture camera trip memory service comp...,,[CLS] terrible camera worse customer service [...,0,7


# Bert-review 多句合併

In [10]:
from copy import deepcopy
def compose_review(x):
    x = eval(x)
    x = "\n".join(x)
    x = x.replace(".\n"," . ").replace("\n."," . ").replace("\n"," ")
#     x = x.replace("\n"," ")    
    tokens = [str(token) for token in x.split(" ") if (" " not in str(token))and (str(token) == '.' or str(token).isalpha())]
    #tokens = [str(token) for token in x.split(" ") if (" " not in str(token)) ]

    #return " ".join(tokens)
    
    newtokens = []
    for token in tokens:
#         if (len(token) == 1 or token == "."):
        if (isnumber(token) or len(token) == 1 or token == ".")and (token not in alphbet_stopword):
            newtokens.append(token)
#             if (token not in ['a','i']) and (token != "."): print(token)
        else:
#             token = token.replace("."," . ")
            token = token.replace("."," . ")
            sub_tokens = token.split(" ")
            sub_tokens = [t for t in sub_tokens if t != "" and t not in alphbet_stopword]
            if len(sub_tokens) == 0: continue
            newtokens.extend(sub_tokens)

    #dot_tokens = [token for token in newtokens if (token[0] == "." or token[-1] == ".") and (len(token)>1)]
    #if len(dot_tokens) > 0 :print(dot_tokens)
    
    return " ".join(newtokens).replace(' . . ',' . ')


'''
def calc_review_len(x):
#     tokens = [token for token in nlp(x)]
#     print(tokens)
#     print([len(t) for t in tokens])
#     return len(tokens)
    return len(x.split(" "))
key_df = deepcopy(orign_key_df)
key_df['lemm_review'] = key_df['lemm_review'].apply(compose_review)
key_df['lemm_review_len'] = key_df['lemm_review'].apply(calc_review_len)
key_df.head()
'''

def bert_compose_review(x):
    x = eval(x)
    review_sents = deepcopy(x)
    total_tokens = []
    for sent in review_sents:
        sent = sent.replace('\n','[SEP]')
        tokens = [str(token) for token in sent.split(" ") if (" " not in str(token)) ]
        newtokens = []
        for token in tokens:
            if (isnumber(token) or len(token) == 1 or token == ".")and (token not in alphbet_stopword):
                newtokens.append(token)
            else:
                token = token.replace("."," . ")
                sub_tokens = token.split(" ")
                sub_tokens = [t for t in sub_tokens if t != "" and t not in alphbet_stopword]
                newtokens.extend(sub_tokens)
        newtokens = newtokens + ['[SEP]']        
        total_tokens.extend(newtokens)
    total_tokens = ['[CLS]'] + tokenizer.tokenize(" ".join(total_tokens))    
    return " ".join(total_tokens)
'''
# def compose_review(bert_review):
#     return bert_review.replace('[CLS] ','').replace('[SEP] ','')
'''
''''''

''

In [11]:
orign_key_df.head()

,review_ID,review,summary,big_categories,small_categories,lemm_review,lemm_summary,lemm_review_len,lemm_summary_len,overall,vote,total_keyword,FOP_sents,total_mention_features,bert_review,bert_summary,bert_review_len,bert_summary_len
0,1356566400,this book was almost as small a pamphlet for 2...,too small,Electronics,Camera Cases,['this book was almost as small a pamphlet for...,<s> too small </s>,44,4,1,2,NaN,NaN,intend small book pamphlet gift expect emphasi...,,[CLS] too small [SEP],0,4
1,1325721600,this book is a collectors item and i love they...,wonderful book a keepsake,Electronics,Camera Cases,['this book is a collector item and i love the...,<s> wonderful book keepsake </s>,53,5,5,9,"item wonderful,quality item,nostalgia wonderfu...",great quality though and a wonderful nostalgia...,item collector book price copy high buy price ...,,[CLS] wonderful book keeps ##ake [SEP],0,6
2,951782400,i would like to say the the cannon elph37oz is...,elph 37oz is the best,Electronics,APS Cameras,['i would like to say the the cannon elph37 oz...,<s> elph 37 oz is the best </s>,58,8,5,51,"camera inferior,money extra,camera cheap",pay the extra money and get this one rather th...,cannon money camera buy decide camera picture ...,,[CLS] el ##ph 37 oz is the best [SEP],0,9
3,985910400,i have had and used q 370z for over a year and...,a good all around aps camera,Electronics,APS Cameras,['i have have and use q 370z for over a year a...,<s> good all around aps camera </s>,137,7,5,13,"eye red,light low room increase lense mm,slrs ...",yes i sometimes get red eye in low light but a...,snapshot photographer year low lens red avoid ...,,[CLS] good all around ap ##s camera [SEP],0,8
4,967075200,i used this camera once and it took relatively...,terrible camera worse customer service,Electronics,APS Cameras,['i use this camera once and it take relativel...,<s> terrible camera worse customer service </s>,96,7,1,48,memory internal camera service camera send sty...,however on my next trip the camera s internal ...,camera picture camera trip memory service comp...,,[CLS] terrible camera worse customer service [...,0,7


In [12]:
key_df = deepcopy(orign_key_df)

from tqdm import tqdm
# 非符號alpha word重疊數
with tqdm(total=len(key_df)) as pbar:
    for i ,row in key_df.iterrows():       
        bert_review = bert_compose_review(row['lemm_review'])
        lemm_review = compose_review(row['lemm_review'])
        
        key_df.loc[i,'bert_review'] = bert_review
        key_df.loc[i,'lemm_review'] = lemm_review
        
        key_df.loc[i,'bert_review_len'] = len(bert_review.split(" "))
        key_df.loc[i,'lemm_review_len'] = len(lemm_review.split(" "))
        
        
        pbar.update(1)
        
amount = len(key_df)
print('Total data : %s'%(amount))

key_df.head()

100%|██████████| 154488/154488 [2:32:05<00:00, 16.93it/s]Total data : 154488



,review_ID,review,summary,big_categories,small_categories,lemm_review,lemm_summary,lemm_review_len,lemm_summary_len,overall,vote,total_keyword,FOP_sents,total_mention_features,bert_review,bert_summary,bert_review_len,bert_summary_len
0,1356566400,this book was almost as small a pamphlet for 2...,too small,Electronics,Camera Cases,this book was almost as small a pamphlet for i...,<s> too small </s>,46,4,1,2,NaN,NaN,intend small book pamphlet gift expect emphasi...,[CLS] this book was almost as small a pamphlet...,[CLS] too small [SEP],51,4
1,1325721600,this book is a collectors item and i love they...,wonderful book a keepsake,Electronics,Camera Cases,this book is a collector item and i love they ...,<s> wonderful book keepsake </s>,56,5,5,9,"item wonderful,quality item,nostalgia wonderfu...",great quality though and a wonderful nostalgia...,item collector book price copy high buy price ...,[CLS] this book is a collector item and i love...,[CLS] wonderful book keeps ##ake [SEP],64,6
2,951782400,i would like to say the the cannon elph37oz is...,elph 37oz is the best,Electronics,APS Cameras,i would like to say the the cannon oz is worth...,<s> elph 37 oz is the best </s>,59,8,5,51,"camera inferior,money extra,camera cheap",pay the extra money and get this one rather th...,cannon money camera buy decide camera picture ...,[CLS] i would like to say the the cannon el ##...,[CLS] el ##ph 37 oz is the best [SEP],69,9
3,985910400,i have had and used q 370z for over a year and...,a good all around aps camera,Electronics,APS Cameras,i have have and use for over a year and i have...,<s> good all around aps camera </s>,135,7,5,13,"eye red,light low room increase lense mm,slrs ...",yes i sometimes get red eye in low light but a...,snapshot photographer year low lens red avoid ...,[CLS] i have have and use 370 ##z for over a y...,[CLS] good all around ap ##s camera [SEP],160,8
4,967075200,i used this camera once and it took relatively...,terrible camera worse customer service,Electronics,APS Cameras,i use this camera once and it take relatively ...,<s> terrible camera worse customer service </s>,99,7,1,48,memory internal camera service camera send sty...,however on my next trip the camera s internal ...,camera picture camera trip memory service comp...,[CLS] i use this camera once and it take relat...,[CLS] terrible camera worse customer service [...,110,7


In [13]:
def to_words(text):
    keywords = set()
    for words in text.split(","):
        for word in words.split(" "):
            keywords.add(word)
    keywords = " ".join(keywords)
    return keywords

def calc_keyword_num(x):
    return len(x.split(" "))

# and(key_df.lemm_review_len>20)
flit_key_df = key_df[(key_df.lemm_summary_len>=4) ] # 過濾single word summary
flit_key_df = flit_key_df[(flit_key_df.lemm_review_len <= 1000) ] # 過濾single word summary
flit_key_df = flit_key_df[(flit_key_df.lemm_review_len >= 50) ] # 過濾single word summary

flit_key_df = flit_key_df.dropna(
    axis=0,     # 0: 对行进行操作; 1: 对列进行操作
    how='any'   # 'any': 只要存在 NaN 就 drop 掉; 'all': 必须全部是 NaN 才 drop 
    )


# 過濾不合適的訓練資料

# FOP_keywords 資料整理

In [14]:
flit_key_df['FOP_keywords'] = flit_key_df['total_keyword']
flit_key_df['FOP_keywords'] = flit_key_df['FOP_keywords'].apply(to_words)
flit_key_df['FOP_keywords_num'] = flit_key_df['FOP_keywords'].apply(calc_keyword_num)
flit_key_df = flit_key_df[(flit_key_df.FOP_keywords_num>=2) ] # 過濾single word summary
flit_key_df.head()

,review_ID,review,summary,big_categories,small_categories,lemm_review,lemm_summary,lemm_review_len,lemm_summary_len,overall,vote,total_keyword,FOP_sents,total_mention_features,bert_review,bert_summary,bert_review_len,bert_summary_len,FOP_keywords,FOP_keywords_num
1,1325721600,this book is a collectors item and i love they...,wonderful book a keepsake,Electronics,Camera Cases,this book is a collector item and i love they ...,<s> wonderful book keepsake </s>,56,5,5,9,"item wonderful,quality item,nostalgia wonderfu...",great quality though and a wonderful nostalgia...,item collector book price copy high buy price ...,[CLS] this book is a collector item and i love...,[CLS] wonderful book keeps ##ake [SEP],64,6,nostalgia wonderful quality great item,5
2,951782400,i would like to say the the cannon elph37oz is...,elph 37oz is the best,Electronics,APS Cameras,i would like to say the the cannon oz is worth...,<s> elph 37 oz is the best </s>,59,8,5,51,"camera inferior,money extra,camera cheap",pay the extra money and get this one rather th...,cannon money camera buy decide camera picture ...,[CLS] i would like to say the the cannon el ##...,[CLS] el ##ph 37 oz is the best [SEP],69,9,inferior camera extra cheap money,5
3,985910400,i have had and used q 370z for over a year and...,a good all around aps camera,Electronics,APS Cameras,i have have and use for over a year and i have...,<s> good all around aps camera </s>,135,7,5,13,"eye red,light low room increase lense mm,slrs ...",yes i sometimes get red eye in low light but a...,snapshot photographer year low lens red avoid ...,[CLS] i have have and use 370 ##z for over a y...,[CLS] good all around ap ##s camera [SEP],160,8,room eye increase light lense mm red slrs low,9
4,967075200,i used this camera once and it took relatively...,terrible camera worse customer service,Electronics,APS Cameras,i use this camera once and it take relatively ...,<s> terrible camera worse customer service </s>,99,7,1,48,memory internal camera service camera send sty...,however on my next trip the camera s internal ...,camera picture camera trip memory service comp...,[CLS] i use this camera once and it take relat...,[CLS] terrible camera worse customer service [...,110,7,internal camera stylus olympus send service me...,9
5,999388800,if you are considering a regular canon elph go...,great purchase,Electronics,APS Cameras,if you are consider a regular canon elph go wi...,<s> great purchase </s>,133,4,5,15,"elph regular,zooming powerful camera lightweig...",if you are consider a regular canon elph go wi...,canon zooming extra elph small print lightweig...,[CLS] if you are consider a regular canon el #...,[CLS] great purchase [SEP],149,4,postcard professional camera elph small lightw...,9


# Cheat Processing

In [15]:
flit_key_df['Cheat'] = False 

# flit_key_df.head()
from tqdm import tqdm
# 非符號alpha word重疊數
with tqdm(total=len(flit_key_df)) as pbar:
    for i ,row in flit_key_df.iterrows():
        rev_tokens = set(row['lemm_review'].split(" "))
        summ_tokens = set(row['lemm_summary'].split(" "))
        key_words = rev_tokens & summ_tokens & (total_keywords| set(opinion_lexicon["total-words"]))
        if len(key_words) > 2: 
            flit_key_df.loc[i,'Cheat'] = True
        pbar.update(1)
    
flit_key_df = flit_key_df[(flit_key_df.Cheat == True) ] # 過濾single word summary
amount = len(flit_key_df)
print('Total data : %s'%(amount))
flit_key_df.head()


100%|██████████| 107132/107132 [03:08<00:00, 568.93it/s]
Total data : 33423


,review_ID,review,summary,big_categories,small_categories,lemm_review,lemm_summary,lemm_review_len,lemm_summary_len,overall,...,total_keyword,FOP_sents,total_mention_features,bert_review,bert_summary,bert_review_len,bert_summary_len,FOP_keywords,FOP_keywords_num,Cheat
4,967075200,i used this camera once and it took relatively...,terrible camera worse customer service,Electronics,APS Cameras,i use this camera once and it take relatively ...,<s> terrible camera worse customer service </s>,99,7,1,...,memory internal camera service camera send sty...,however on my next trip the camera s internal ...,camera picture camera trip memory service comp...,[CLS] i use this camera once and it take relat...,[CLS] terrible camera worse customer service [...,110,7,internal camera stylus olympus send service me...,9,True
8,951868800,my review will focus on how well the 370z repl...,a good enough replacement for 35mm point and s...,Electronics,APS Cameras,my review will focus on how well the replace a...,<s> good enough replacement for 35 mm point an...,419,11,4,...,resolution indistinguishable definition precis...,resolution indistinguishable on 4x6 4x7 print ...,shoot point replace mm review focus replace si...,[CLS] my review will focus on how well the 370...,[CLS] good enough replacement for 35 mm point ...,525,11,continuous light special activity definition c...,29,True
10,974332800,if you want a point and shoot camera that you ...,best compact aps camera,Electronics,APS Cameras,if you want a point and shoot camera that you ...,<s> best compact aps camera </s>,188,6,5,...,"range zoon,range size,size zoon,set pretty bro...",i look at a pretty broad set of camera include...,shoot throw carry point bag camera shirt pocke...,[CLS] if you want a point and shoot camera tha...,[CLS] best compact ap ##s camera [SEP],209,7,picture pretty size camera elph broad surprisi...,13,True
35,942883200,i have had my camera for two months and i love...,great first digital camera for non photographers,Electronics,Digital Cameras,i have have my camera for two month and i love...,<s> great first digital camera for non photogr...,105,9,4,...,"quality good,quality very good,mb card",i wish that it would take the 16 mb and the 32...,month camera photographer professional picture...,[CLS] i have have my camera for two month and ...,[CLS] great first digital camera for non photo...,117,9,very quality mb card good,5,True
44,960163200,when i got this camera it is nice looking and ...,use 3 days to trouble shoot the connecting the...,Electronics,Digital Cameras,when i get this camera it is nice looking and ...,<s> use day to trouble shoot the connect the s...,190,12,3,...,buying worth picture nice photo upload cable s...,when i get this camera it is nice looking and ...,camera buying picture day photo computer uploa...,[CLS] when i get this camera it is nice lookin...,[CLS] use day to trouble shoot the connect the...,221,12,worth photo instal picture buying nice softwar...,11,True


# TextRank_keywords 資料整理

In [16]:
from summa import keywords as TextRank
from summa.summarizer import summarize
def textrank_keys(text):
    keywords1 = list()
    for words in TextRank.keywords(text).split('\n'):
        keywords1.extend(words.split(" "))
    keywords1 = set(keywords1)    
    
    return " ".join(list(keywords1))

def textrank_summ_keys(text): 
    keywords2 = list()
    for words in summarize(text, words=8).split('\n'):
        keywords2.extend(words.split(" "))
    keywords2 = set(keywords2)
    
    return " ".join(list(keywords2))

# flit_key_df['TextRank_keywords'] = flit_key_df['lemm_review'].apply(textrank_to_words)
# flit_key_df['TextRank_keywords_num'] = flit_key_df['TextRank_keywords'].apply(calc_num)
flit_key_df['TextRank_keywords'] = flit_key_df['FOP_keywords'] 
flit_key_df['TextRank_keywords_num'] = flit_key_df['FOP_keywords_num'] 
flit_key_df.loc[:,'TextRank_keywords'] = ''
flit_key_df.loc[:,'TextRank_keywords_num'] = 0
# flit_key_df.head()



In [17]:
from tqdm import tqdm
with tqdm(total=len(flit_key_df)) as pbar:
    for i ,row in flit_key_df.iterrows():
        TextRank_keywords = textrank_keys(row['lemm_review'])
    #     TextRank_keywords = textrank_summ_keys(row['lemm_review'])  
        num = calc_keyword_num(TextRank_keywords)
        flit_key_df.loc[i,'TextRank_keywords'] = TextRank_keywords
        flit_key_df.loc[i,'TextRank_keywords_num'] = num
        pbar.update(1)
        
flit_key_df.head()

100%|██████████| 33423/33423 [08:37<00:00, 64.53it/s]


,review_ID,review,summary,big_categories,small_categories,lemm_review,lemm_summary,lemm_review_len,lemm_summary_len,overall,...,total_mention_features,bert_review,bert_summary,bert_review_len,bert_summary_len,FOP_keywords,FOP_keywords_num,Cheat,TextRank_keywords,TextRank_keywords_num
4,967075200,i used this camera once and it took relatively...,terrible camera worse customer service,Electronics,APS Cameras,i use this camera once and it take relatively ...,<s> terrible camera worse customer service </s>,99,7,1,...,camera picture camera trip memory service comp...,[CLS] i use this camera once and it take relat...,[CLS] terrible camera worse customer service [...,110,7,internal camera stylus olympus send service me...,9,True,equipment good camera,3
8,951868800,my review will focus on how well the 370z repl...,a good enough replacement for 35mm point and s...,Electronics,APS Cameras,my review will focus on how well the replace a...,<s> good enough replacement for 35 mm point an...,419,11,4,...,shoot point replace mm review focus replace si...,[CLS] my review will focus on how well the 370...,[CLS] good enough replacement for 35 mm point ...,525,11,continuous light special activity definition c...,29,True,day excellent eye frame precise use mm wishlis...,16
10,974332800,if you want a point and shoot camera that you ...,best compact aps camera,Electronics,APS Cameras,if you want a point and shoot camera that you ...,<s> best compact aps camera </s>,188,6,5,...,shoot throw carry point bag camera shirt pocke...,[CLS] if you want a point and shoot camera tha...,[CLS] best compact ap ##s camera [SEP],209,7,picture pretty size camera elph broad surprisi...,13,True,shoot camera elph broad pocket shirt expensive...,9
35,942883200,i have had my camera for two months and i love...,great first digital camera for non photographers,Electronics,Digital Cameras,i have have my camera for two month and i love...,<s> great first digital camera for non photogr...,105,9,4,...,month camera photographer professional picture...,[CLS] i have have my camera for two month and ...,[CLS] great first digital camera for non photo...,117,9,very quality mb card good,5,True,photographer professional quality,3
44,960163200,when i got this camera it is nice looking and ...,use 3 days to trouble shoot the connecting the...,Electronics,Digital Cameras,when i get this camera it is nice looking and ...,<s> use day to trouble shoot the connect the s...,190,12,3,...,camera buying picture day photo computer uploa...,[CLS] when i get this camera it is nice lookin...,[CLS] use day to trouble shoot the connect the...,221,12,worth photo instal picture buying nice softwar...,11,True,worth way buying picture nice looking,6


In [18]:
from matplotlib import pyplot as plt
if not os.path.exists('XLSX/statistic'):
    os.makedirs('XLSX/statistic')
with open('XLSX/statistic/%s_%s_info.txt'%(category1,category2),'w') as f:
    max_rev_len = flit_key_df['lemm_review_len'].max()
    min_rev_len = flit_key_df['lemm_review_len'].min()
    mean_rev_len = flit_key_df['lemm_review_len'].mean()
    median_rev_len = flit_key_df['lemm_review_len'].median()

    f.write('max_rev_len :%s \n'%(max_rev_len))
    f.write('min_rev_len :%s \n'%(min_rev_len))
    f.write('mean_rev_len :%s \n'%(mean_rev_len))
    f.write('median_rev_len :%s \n'%(median_rev_len))
    
    f.write('\n\n\n')
    max_summary_len = flit_key_df['lemm_summary_len'].max()
    min_summary_len = flit_key_df['lemm_summary_len'].min()
    mean_summary_len = flit_key_df['lemm_summary_len'].mean()
    median_summary_len = flit_key_df['lemm_summary_len'].median()

    f.write('max_summary_len :%s \n'%(max_summary_len))
    f.write('min_summary_len :%s \n'%(min_summary_len))
    f.write('mean_summary_len :%s \n'%(mean_summary_len))
    f.write('median_summary_len :%s \n'%(median_summary_len))
    
    f.write('\n\n\n')
    max_FOP_keywords_num = flit_key_df['FOP_keywords_num'].max()
    min_FOP_keywords_num = flit_key_df['FOP_keywords_num'].min()
    mean_FOP_keywords_num = flit_key_df['FOP_keywords_num'].mean()
    median_FOP_keywords_num = flit_key_df['FOP_keywords_num'].median()

    f.write('max_FOP_keywords_num :%s \n'%(max_FOP_keywords_num))
    f.write('min_FOP_keywords_num :%s \n'%(min_FOP_keywords_num))
    f.write('mean_FOP_keywords_num :%s \n'%(mean_FOP_keywords_num))
    f.write('median_FOP_keywords_num :%s \n'%(median_FOP_keywords_num))
    
    f.write('\n\n\n')
    max_TextRank_keywords_num = flit_key_df['TextRank_keywords_num'].max()
    min_TextRank_keywords_num = flit_key_df['TextRank_keywords_num'].min()
    mean_TextRank_keywords_num = flit_key_df['TextRank_keywords_num'].mean()
    median_TextRank_keywords_num = flit_key_df['TextRank_keywords_num'].median()

    f.write('max_TextRank_keywords_num :%s \n'%(max_TextRank_keywords_num))
    f.write('min_TextRank_keywords_num :%s \n'%(min_TextRank_keywords_num))
    f.write('mean_TextRank_keywords_num :%s \n'%(mean_TextRank_keywords_num))
    f.write('median_TextRank_keywords_num :%s \n'%(median_TextRank_keywords_num))

    f.write('\n\n\n')    
    max_bert_rev_len = flit_key_df['bert_review_len'].max()
    min_bert_rev_len = flit_key_df['bert_review_len'].min()
    mean_bert_rev_len = flit_key_df['bert_review_len'].mean()
    median_bert_rev_len = flit_key_df['bert_review_len'].median()

    f.write('max_bert_rev_len :%s \n'%(max_bert_rev_len))
    f.write('min_bert_rev_len :%s \n'%(min_bert_rev_len))
    f.write('mean_bert_rev_len :%s \n'%(mean_bert_rev_len))
    f.write('median_bert_rev_len :%s \n'%(median_bert_rev_len))
    
    f.write('\n\n\n')
    max_bert_summary_len = flit_key_df['bert_summary_len'].max()
    min_bert_summary_len = flit_key_df['bert_summary_len'].min()
    mean_bert_summary_len = flit_key_df['bert_summary_len'].mean()
    median_bert_summary_len = flit_key_df['bert_summary_len'].median()

    f.write('max_bert_summary_len :%s \n'%(max_bert_summary_len))
    f.write('min_bert_summary_len :%s \n'%(min_bert_summary_len))
    f.write('mean_bert_summary_len :%s \n'%(mean_bert_summary_len))
    f.write('median_bert_summary_len :%s \n'%(median_bert_summary_len))

    

# plt.xlim(xmax = mean_rev_len)
# plt.ylim(ymax = flit_key_df['lemm_review_len'].value_counts().max())

flit_key_df['lemm_review_len'].value_counts().hist()
plt.savefig('XLSX/statistic/review_len_%s_%s.png'%(category1,category2))
plt.show()
plt.close()

# plt.xlim(xmax = max_summary_len)
# plt.ylim(ymax = flit_key_df['lemm_summary_len'].value_counts().max())
flit_key_df['lemm_summary_len'].value_counts().hist()
plt.savefig('XLSX/statistic/summary_len_%s_%s.png'%(category1,category2))
plt.show()
plt.close()

# plt.xlim(xmax = mean_keyword_num)
flit_key_df['FOP_keywords_num'].value_counts().hist()
plt.savefig('XLSX/statistic/FOP_keywords_num_%s_%s.png'%(category1,category2))
plt.show()
plt.close()

flit_key_df['TextRank_keywords_num'].value_counts().hist()
plt.savefig('XLSX/statistic/TextRank_keywords_num_%s_%s.png'%(category1,category2))
plt.show()
plt.close()


<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

# 製作record bin檔

In [38]:
import shutil
if not os.path.exists('bin'):
    shutil.rmtree('/bin', ignore_errors=True)

if not os.path.exists('bin/category/chunked'):
    os.makedirs('bin/category/chunked')

makevocab = True
if makevocab:
    vocab_counter = collections.Counter()
    
# train_file
flit_key_train_df = flit_key_df.iloc[:int(amount*0.6)]

# test_file
flit_key_test_df = flit_key_df.iloc[int(amount*0.6)+1:int(amount*0.8)]

# vald_file
flit_key_valid_df = flit_key_df.iloc[int(amount*0.8)+1:]
sentence_start = "<s>"
sentence_end = "</s>"


def xlsx2bin(set_name,df):
    sents = []
    with open("bin/category/%s.bin"%(set_name), 'wb') as file:
        i = 0
        for idx in tqdm(range(len(df))):
            series = df.iloc[idx]
            data_dict = series.to_dict()
#             review_ID , big_categories , small_categories , \
#             review , lemm_review , summary , lemm_summary , FOP_keywords ,TextRank_keywords = \
#             data_dict['review_ID'],data_dict['big_categories'],data_dict['small_categories'],data_dict['review'],data_dict['lemm_review'], \
#             data_dict['summary'],data_dict['lemm_summary'],data_dict['FOP_keywords'] ,data_dict['TextRank_keywords']


            
            review_ID , big_categories , small_categories , \
            orign_review , lemm_review , orign_summary , lemm_summary , \
            bert_review , bert_summary ,FOP_keywords ,TextRank_keywords = \
            data_dict['review_ID'],data_dict['big_categories'],data_dict['small_categories'], \
            data_dict['review'],data_dict['lemm_review'], data_dict['summary'],data_dict['lemm_summary'], \
            data_dict['bert_review'], data_dict['bert_summary'] , data_dict['FOP_keywords'] ,data_dict['TextRank_keywords']
            
#             print(FOP_keywords)

            # save Embedding/word2Vec calculate sents
#             for sent in nltk.sent_tokenize(lemm_review):
#                 sent = sent.replace("." ,"")
#                 sents.append(str(sent).split()) # 切分词汇 

#             for sent in nltk.sent_tokenize(lemm_summary):
#                 sent = sent.replace(sentence_start ,"").replace(sentence_end ,"")
#                 sents.append(str(sent).split()) # 切分词汇 

            lemm_review = lemm_review.replace("\n","")
            lemm_summary = lemm_summary.replace("\n","").replace("."," ")
            # lemm_summary = sentence_start + ' '+ lemm_summary + ' ' + sentence_end
#             print(lemm_summary)
            # Write to tf.Example
            tf_example = example_pb2.Example()
            try:
                tf_example.features.feature['orign_review'].bytes_list.value.extend(
                    [tf.compat.as_bytes(orign_review, encoding='utf-8')])

                tf_example.features.feature['orign_summary'].bytes_list.value.extend(
                    [tf.compat.as_bytes(orign_summary, encoding='utf-8')])
                
                tf_example.features.feature['review'].bytes_list.value.extend(
                    [tf.compat.as_bytes(lemm_review, encoding='utf-8')])

                tf_example.features.feature['summary'].bytes_list.value.extend(
                    [tf.compat.as_bytes(lemm_summary, encoding='utf-8')]) 
                    
                tf_example.features.feature['bert_review'].bytes_list.value.extend(
                    [tf.compat.as_bytes(bert_review, encoding='utf-8')])

                tf_example.features.feature['bert_summary'].bytes_list.value.extend(
                    [tf.compat.as_bytes(bert_summary, encoding='utf-8')])
        
                tf_example.features.feature['FOP_keywords'].bytes_list.value.extend(
                    [tf.compat.as_bytes(FOP_keywords, encoding='utf-8')]) 
            
                tf_example.features.feature['TextRank_keywords'].bytes_list.value.extend(
                    [tf.compat.as_bytes(TextRank_keywords, encoding='utf-8')]) 

                tf_example_str = tf_example.SerializeToString()
                str_len = len(tf_example_str)  
                file.write(struct.pack('q', str_len))
                file.write(struct.pack('%ds' % str_len, tf_example_str))
            except Exception as e:
                print(e)
                pass
    print(" %s finished... "%(file.name))
    return sents
    
    
sents1 = xlsx2bin('train',flit_key_train_df)
sents2 = xlsx2bin('test',flit_key_test_df)
sents3 = xlsx2bin('valid',flit_key_valid_df)

100%|██████████| 6684/6684 [00:02<00:00, 3230.21it/s] bin/category/valid.bin finished... 



In [39]:
with open("bin/category/bin-info.txt",'w',encoding='utf-8') as f :
    f.write("train : %s\n"%(len(flit_key_train_df)))
    f.write("test : %s\n"%(len(flit_key_test_df)))
    f.write("valid : %s\n"%(len(flit_key_valid_df)))

# 分割record bin檔(1000為單位)

In [40]:
def chunk_file(set_name, chunks_dir):
    in_file = 'bin/category/%s.bin' % set_name
    reader = open(in_file, "rb")
    chunk = 0
    finished = False
    while not finished:
#         chunk_fname = os.path.join('bin', '/%s/%s_%03d.bin' % (chunks_dir,set_name, chunk))  # new chunk
        chunk_fname = '%s/%s/%s_%03d.bin' % (chunks_dir,set_name,set_name, chunk)
        with open(chunk_fname, 'wb') as writer:
            for _ in range(CHUNK_SIZE):
                len_bytes = reader.read(8)
                if not len_bytes:
                    finished = True
                    break
                str_len = struct.unpack('q', len_bytes)[0]
                example_str = struct.unpack('%ds' % str_len, reader.read(str_len))[0]
                writer.write(struct.pack('q', str_len))
                writer.write(struct.pack('%ds' % str_len, example_str))
            chunk += 1


def chunk_all(chunks_dir = 'bin/category/chunked'):
    # Make a dir to hold the chunks
    
    # Chunk the data
    for set_name in ['train', 'valid', 'test']:
        if not os.path.isdir(os.path.join(chunks_dir,set_name)):
            os.mkdir(os.path.join(chunks_dir,set_name))
        print("Splitting %s data into chunks..." % set_name)
        chunk_file(set_name, chunks_dir)
    print("Saved chunked data in %s" % chunks_dir)
    
chunk_all()

Splitting train data into chunks...
Splitting valid data into chunks...
Splitting test data into chunks...
Saved chunked data in bin/category/chunked


In [41]:
def main_valid():
    #Performing rouge evaluation on 1.9 lakh sentences takes lot of time. So, create mini validation set & test set by borrowing 15k samples each from these 1.9 lakh sentences
    bin_valid_chuncks = os.listdir('bin/category/chunked/valid/')
    bin_valid_chuncks.sort()
    if not os.path.exists('bin/category/chunked/main_valid'):
        os.makedirs('bin/category/chunked/main_valid')
        
    samples = random.sample(set(bin_valid_chuncks[:-1]), 2)      #Exclude last bin file; contains only 9k sentences
    valid_chunk, test_chunk = samples[0], samples[1]
    shutil.copyfile(os.path.join('bin/category/chunked/valid', valid_chunk), os.path.join("bin/category/chunked/main_valid", "valid_00.bin"))
    shutil.copyfile(os.path.join('bin/category/chunked/valid', test_chunk), os.path.join("bin/category/chunked/main_valid", "test_00.bin"))
main_valid()

# Embedding/word2Vec

In [23]:
sentences = [] # total sentence
for idx in tqdm(range(len(orign_key_df))):
    series = orign_key_df.iloc[idx]
    data_dict = series.to_dict()
    lemm_review_sents , lemm_summary  = data_dict['lemm_review'],data_dict['lemm_summary'] 
    lemm_review_sents = eval(lemm_review_sents)
    for sent in lemm_review_sents:
        sent_tokens = sent.split(" ")
        tokens = [str(token) for token in sent.split() if (" " not in str(token))and (str(token) == '.' or str(token).isalpha())]
#         tokens = [str(token) for token in sent.split() if (" " not in str(token))]
        sentences.append(tokens)   
    
        dot_tokens = [token for token in tokens if (token[0] == "." or token[-1] == ".") and (len(token)>1)]
        if len(dot_tokens) > 0 :print(dot_tokens)
        
    sentences.append([t for t in lemm_summary.split(" ") if t not in ["<s>" , "</s>"]])
    
    dot_tokens = [token for token in lemm_summary.split(" ") if (token[0] == "." or token[-1] == ".") and (len(token)>1)]
    if len(dot_tokens) > 0 :print(dot_tokens)
    
print('word2Vec training sentence finished...')

100%|██████████| 154488/154488 [00:58<00:00, 2622.56it/s]word2Vec training sentence finished...



In [24]:
# 引入 word2vec
from gensim.models import word2vec
from glob import glob
import sys

import gensim
import torch
import torch.nn as nn
import torchsnooper
import os
import numpy as np

# 引入日志配置
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

vocab_count = 50000


In [25]:
# write vocab to file
if not os.path.exists('Embedding/category/word2Vec'):
    os.makedirs('Embedding/category/word2Vec')
    
if not os.path.exists("Embedding/category/word2Vec/word2Vec.300d.txt"):

    w2vec = word2vec.Word2Vec(sentences, size=300, min_count=2,max_vocab_size=None,iter=100,
                              sorted_vocab=1,max_final_vocab=vocab_count)

    

    w2vec.wv.save_word2vec_format('Embedding/category/word2Vec/word2Vec.300d.txt', binary=False)

    #保存模型，供日後使用
    # w2vec.save("Embedding/word2Vec/word2vec.model")   


: EPOCH 93 - PROGRESS: at 50.54% examples, 1036549 words/s, in_qsize 5, out_qsize 0
2020-01-27 05:44:27,772 : INFO : EPOCH 93 - PROGRESS: at 55.27% examples, 1033447 words/s, in_qsize 5, out_qsize 0
2020-01-27 05:44:28,774 : INFO : EPOCH 93 - PROGRESS: at 60.02% examples, 1029403 words/s, in_qsize 5, out_qsize 0
2020-01-27 05:44:29,785 : INFO : EPOCH 93 - PROGRESS: at 64.90% examples, 1028186 words/s, in_qsize 5, out_qsize 0
2020-01-27 05:44:30,787 : INFO : EPOCH 93 - PROGRESS: at 70.13% examples, 1029869 words/s, in_qsize 5, out_qsize 0
2020-01-27 05:44:31,791 : INFO : EPOCH 93 - PROGRESS: at 75.32% examples, 1032123 words/s, in_qsize 5, out_qsize 0
2020-01-27 05:44:32,796 : INFO : EPOCH 93 - PROGRESS: at 80.50% examples, 1031826 words/s, in_qsize 5, out_qsize 0
2020-01-27 05:44:33,797 : INFO : EPOCH 93 - PROGRESS: at 85.75% examples, 1032222 words/s, in_qsize 5, out_qsize 0
2020-01-27 05:44:34,798 : INFO : EPOCH 93 - PROGRESS: at 90.83% examples, 1029597 words/s, in_qsize 5, out_qsiz

In [26]:
# sentences = sents1 + sents2 + sents3

In [27]:
#模型讀取方式
# model = word2vec.Word2Vec.load("Embedding/word2Vec/word2vec.model")

wvmodel = gensim.models.KeyedVectors.load_word2vec_format(
    'Embedding/category/word2Vec/word2Vec.300d.txt', binary=False, encoding='utf-8')

wvmodel.most_similar(u"player", topn=10)
# wvmodel.most_similar(['dvd','player','changer','machine','video'], topn=20)

2020-01-27 05:47:00,644 : INFO : loading projection weights from Embedding/category/word2Vec/word2Vec.300d.txt
2020-01-27 05:47:08,619 : INFO : loaded (39048, 300) matrix from Embedding/category/word2Vec/word2Vec.300d.txt
2020-01-27 05:47:08,621 : INFO : precomputing L2-norms of word weight vectors


[('vlc', 0.44760793447494507),
 ('quicktime', 0.4168507158756256),
 ('tv', 0.41560834646224976),
 ('editor', 0.4100143611431122),
 ('microsoft', 0.4048650860786438),
 ('avi', 0.39370083808898926),
 ('realplayer', 0.3936995267868042),
 ('application', 0.39163148403167725),
 ('wma', 0.39099571108818054),
 ('dvd', 0.3842791020870209)]

In [28]:
vocab_file = "Embedding/category/word2Vec/word.vocab"

if not os.path.exists(vocab_file):
    vocab_count = len(wvmodel.wv.index2entity)    

    print("Writing vocab file...")
    with open(vocab_file, 'w',encoding='utf-8') as writer:
        for word in wvmodel.wv.index2entity[:vocab_count]:
            # print(word, w2vec.wv.vocab[word].count)
            writer.write(word + ' ' + str(wvmodel.wv.vocab[word].count) + '\n') # Output vocab count
    print("Finished writing vocab file")

/home/eagleuser/.conda/envs/Leyan/lib/python3.6/site-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  after removing the cwd from sys.path.
/home/eagleuser/.conda/envs/Leyan/lib/python3.6/site-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  
/home/eagleuser/.conda/envs/Leyan/lib/python3.6/site-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  # Remove the CWD from sys.path while we load stuff.
Writing vocab file...
Finished writing vocab file


In [29]:
word = wvmodel.wv.index2entity[25]
vector = wvmodel.wv.vectors[25]
print(word)
# print(vector)

take
/home/eagleuser/.conda/envs/Leyan/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.
/home/eagleuser/.conda/envs/Leyan/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  


In [30]:
import torch
from data_util.data import Vocab
vocab_size = len(wvmodel.vocab) + 1


vocab = Vocab('Embedding/category/word2Vec/word.vocab', vocab_size)

embed_size = 300
weight = torch.zeros(vocab_size, embed_size)

for i in range(len(vocab._id_to_word.keys())):
    try:
        vocab_word = vocab._id_to_word[i+4]
        w2vec_word = w2vec.wv.index2entity[i]
    except Exception as e :
        continue
    if i + 4 > vocab_size: break
#     print(vocab_word,w2vec_word)
    weight[i+4, :] = torch.from_numpy(w2vec.wv.vectors[i])
        
embedding = torch.nn.Embedding.from_pretrained(weight)
# requires_grad指定是否在训练过程中对词向量的权重进行微调
embedding.weight.requires_grad = True
embedding

Embedding(39049, 300)

In [31]:
vocab.word2id('the')

4

# Embedding/glove

In [32]:
from glove import Glove
from glove import Corpus

vocab_count = 50000
# write vocab to file
if not os.path.exists('Embedding/category/glove'):
    os.makedirs('Embedding/category/glove')

In [33]:
if not os.path.exists("Embedding/category/glove/glove.model"):

    corpus_model = Corpus()
    corpus_model.fit(sentences, window=10)
    #corpus_model.save('corpus.model')
    print('Dict size: %s' % len(corpus_model.dictionary))
    print('Collocations: %s' % corpus_model.matrix.nnz)
    
    glove = Glove(no_components=300, learning_rate=0.05)
    glove.fit(corpus_model.matrix, epochs=100,
              no_threads=10, verbose=True)
    glove.add_dictionary(corpus_model.dictionary)
    
    glove.save('Embedding/category/glove/glove.model') # 存模型
    corpus_model.save('Embedding/category/glove/corpus.model') # 存字典


glove = Glove.load('Embedding/category/glove/glove.model')
corpus_model = Corpus.load('Embedding/category/glove/corpus.model')

Dict size: 70986
Collocations: 9370642
Performing 100 training epochs with 10 threads
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
Epoch 20
Epoch 21
Epoch 22
Epoch 23
Epoch 24
Epoch 25
Epoch 26
Epoch 27
Epoch 28
Epoch 29
Epoch 30
Epoch 31
Epoch 32
Epoch 33
Epoch 34
Epoch 35
Epoch 36
Epoch 37
Epoch 38
Epoch 39
Epoch 40
Epoch 41
Epoch 42
Epoch 43
Epoch 44
Epoch 45
Epoch 46
Epoch 47
Epoch 48
Epoch 49
Epoch 50
Epoch 51
Epoch 52
Epoch 53
Epoch 54
Epoch 55
Epoch 56
Epoch 57
Epoch 58
Epoch 59
Epoch 60
Epoch 61
Epoch 62
Epoch 63
Epoch 64
Epoch 65
Epoch 66
Epoch 67
Epoch 68
Epoch 69
Epoch 70
Epoch 71
Epoch 72
Epoch 73
Epoch 74
Epoch 75
Epoch 76
Epoch 77
Epoch 78
Epoch 79
Epoch 80
Epoch 81
Epoch 82
Epoch 83
Epoch 84
Epoch 85
Epoch 86
Epoch 87
Epoch 88
Epoch 89
Epoch 90
Epoch 91
Epoch 92
Epoch 93
Epoch 94
Epoch 95
Epoch 96
Epoch 97
Epoch 98
Epoch 99


In [34]:
vocab_file = "Embedding/category/glove/word.vocab"

if not os.path.exists(vocab_file):
#     vocab_count = len(glove.dictionary)    
    vocab_count = 0
    print("Writing vocab file...")
    with open(vocab_file, 'w',encoding='utf-8') as writer:
        for word,idx in glove.dictionary.items():
            if word in vocab._word_to_id.keys():
                vocab_count += 1
                writer.write(word + ' ' + str(idx) + '\n') # Output vocab count
    print("Finished writing vocab file %s" %(vocab_count))

Writing vocab file...
Finished writing vocab file 39045


In [35]:
glove.word_vectors[glove.dictionary['.']].shape
# vocab._word_to_id.keys()
# len(glove.dictionary)

(300,)

In [36]:
vocab_size = len(open('Embedding/category/glove/word.vocab').readlines())
print(vocab_size)

vocab = Vocab('Embedding/category/glove/word.vocab', vocab_size)
embed_size = 300
weight = torch.zeros(vocab_size, embed_size)

for word,idx in glove.dictionary.items():
    if word in vocab._word_to_id.keys():
        wid = vocab.word2id(word) 
        vector = np.asarray(glove.word_vectors[glove.dictionary[word]], "float32")
        weight[wid, :] = torch.from_numpy(vector)

embedding = torch.nn.Embedding.from_pretrained(weight)
# requires_grad指定是否在训练过程中对词向量的权重进行微调
embedding.weight.requires_grad = True
embedding 

39045


Embedding(39045, 300)

# Embedding/Bert

In [37]:
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM
# BERT
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True, do_basic_tokenize=True)
# Load pre-trained model (weights)
model = BertModel.from_pretrained('bert-base-uncased')
# model.eval()
model.embeddings.word_embeddings


# vocab = Vocab('Embedding/word2Vec/word2Vec.vocab', vocab_size)

# embed_size = 300
# weight = torch.zeros(vocab_size, embed_size)


# embedding = torch.nn.Embedding.from_pretrained(weight)
# # requires_grad指定是否在训练过程中对词向量的权重进行微调
# embedding.weight.requires_grad = True
# embedding        

2020-01-27 06:03:20,921 : INFO : loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz from cache at /home/eagleuser/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba
2020-01-27 06:03:20,927 : INFO : extracting archive file /home/eagleuser/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba to temp dir /tmp/tmpvk3vkdaw
2020-01-27 06:03:23,655 : INFO : Model config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "type_vocab_size": 2,
  "vocab_size": 30522
}



Embedding(30522, 768, padding_idx=0)